In [1]:
import torch
import numpy
from torch import nn
from torch.nn import functional as TF

Step1: 定义常量

In [2]:
# Batch大小
BATCH_SIZE = 2
# 单词表中的单词总数
VOCABULARY_SIZE = 8
# 句子的最大长度
MAX_SENTENCE_LEN = 5

In [3]:
# 模型维度（特征维度）大小
MODEL_DIM = 8  # 原论文是512，演示起见，这里定义成8

Step2: 演示起见，手写训练数据集

In [4]:
"""生成一个batch的随机句子当做训练集
   句子由token组成，有独热编码和词袋模型两种表示token的方式
   这里采用词袋模型，也就是一个词用一个独特的整数表示
"""

# Batch size为2，也就是2个句子
# 输入的句子长度分别为2,4
train_input_lengths = torch.Tensor([2, 4]).to(torch.int32)
# 标签的句子长度分别为4,3
train_label_lengths = torch.Tensor([4, 3]).to(torch.int32)

train_input_sentences = []
for length in train_input_lengths:
    # 按照长度随机生成句子
    input = torch.randint(1, VOCABULARY_SIZE, (length,))
    # 将句子padding到最大长度
    input = TF.pad(input, (0, MAX_SENTENCE_LEN - length))
    train_input_sentences.append(input)

train_label_sentences = []
for length in train_label_lengths:
    # 按照长度随机生成句子
    label = torch.randint(1, VOCABULARY_SIZE, (length,))
    # 将句子padding到最大长度
    label = TF.pad(label, (0, MAX_SENTENCE_LEN - length))
    train_label_sentences.append(label)

print(f"训练集输入句子：{train_input_sentences}")
print(f"训练集标签句子：{train_label_sentences}")

训练集输入句子：[tensor([2, 3, 0, 0, 0]), tensor([7, 1, 6, 7, 0])]
训练集标签句子：[tensor([2, 7, 5, 1, 0]), tensor([1, 7, 7, 0, 0])]


In [5]:
"""把句子列表变成矩阵
   把每个句子先变成二维矩阵，再拼接到一起成为真正的输入矩阵和标签矩阵
"""

for index, value in enumerate(train_input_sentences):
    value = torch.squeeze(value)
    train_input_sentences[index] = torch.unsqueeze(value, dim=0)

for index, value in enumerate(train_label_sentences):
    value = torch.squeeze(value)
    train_label_sentences[index] = torch.unsqueeze(value, dim=0)

train_input_mat = torch.cat(train_label_sentences, dim=0)
train_label_mat = torch.cat(train_label_sentences, dim=0)

print(f"训练集输入矩阵：\n{train_input_mat}")
print(f"训练集标签矩阵：\n{train_label_mat}")

训练集输入矩阵：
tensor([[2, 7, 5, 1, 0],
        [1, 7, 7, 0, 0]])
训练集标签矩阵：
tensor([[2, 7, 5, 1, 0],
        [1, 7, 7, 0, 0]])


Step3: 构造Embedding

In [6]:
# 词汇表大小+1是因为多了个pad出来的0
input_embedding_layer = nn.Embedding(VOCABULARY_SIZE + 1, MODEL_DIM)
lable_embedding_layer = nn.Embedding(VOCABULARY_SIZE + 1, MODEL_DIM)

train_input_embedding = input_embedding_layer(train_input_mat)
train_label_embedding = lable_embedding_layer(train_label_mat)

print(f"训练集输入Embedding：\n{train_input_embedding}")

训练集输入Embedding：
tensor([[[ 1.4604, -0.4342, -1.2865, -1.6023,  1.0133, -0.8629, -2.4131,
           0.6980],
         [-0.3962,  0.8665,  2.0806, -0.8865, -0.3765,  0.5652, -0.0443,
          -0.0767],
         [-0.6952, -0.0707, -1.7628,  0.2006,  1.5358,  1.1774,  0.3667,
           0.2336],
         [ 1.7606,  0.1262,  0.1185, -0.3747, -1.8714,  1.4624,  0.1591,
          -1.6482],
         [-0.7516, -0.1521, -0.8365, -0.5776, -0.3457, -0.8162,  1.6633,
           1.0636]],

        [[ 1.7606,  0.1262,  0.1185, -0.3747, -1.8714,  1.4624,  0.1591,
          -1.6482],
         [-0.3962,  0.8665,  2.0806, -0.8865, -0.3765,  0.5652, -0.0443,
          -0.0767],
         [-0.3962,  0.8665,  2.0806, -0.8865, -0.3765,  0.5652, -0.0443,
          -0.0767],
         [-0.7516, -0.1521, -0.8365, -0.5776, -0.3457, -0.8162,  1.6633,
           1.0636],
         [-0.7516, -0.1521, -0.8365, -0.5776, -0.3457, -0.8162,  1.6633,
           1.0636]]], grad_fn=<EmbeddingBackward0>)


Step4: 位置编码

正弦/余弦位置编码：
每个维度都是独特的值，偶数维度用sin，奇数维度用cos：
$$
\begin{equation}
\begin{split}
\text{PE}_{pos, 2i} &= \sin{(pos/10000^{2i/d_{\text{model}}})} \\
\text{PE}_{pos, 2i+1} &= \cos{(pos/10000^{2i/d_{\text{model}}})}
\end{split}
\end{equation}
$$
其中，$pos$代表token位置索引，$i$用来表示位置编码的维度索引，$2i$表示偶数位置，$2i+1$表示奇数位置。

加上位置矩阵后，同样的词组成的句子，词的语序不一样，那么词的embedding也不一样，这就成功带上了语序信息。

In [7]:
pos_mat = torch.arange(MAX_SENTENCE_LEN).reshape((-1, 1))
dim_mat = torch.arange(MODEL_DIM).reshape((1, -1))
print(f"pos matrix:\n{pos_mat}\n")
print(f"dim matrix:\n{dim_mat}\n")

# 这里利用了Python的广播机制，最终得到了一个(MAX_SENTENCE_LEN, MODEL_DIM)的矩阵
para = pos_mat / torch.pow(10000, dim_mat / MODEL_DIM)

pe_mat = torch.zeros(MAX_SENTENCE_LEN, MODEL_DIM)
# 偶数维度取偶数列
pe_mat[:, 0::2] = torch.sin(para[:, 0::2])
# 奇数维度取奇数列
pe_mat[:, 1::2] = torch.cos(para[:, 1::2])

# 得到的就是一句样本对应的PE
print(f"Position Encoding:\n{pe_mat}")

pos matrix:
tensor([[0],
        [1],
        [2],
        [3],
        [4]])

dim matrix:
tensor([[0, 1, 2, 3, 4, 5, 6, 7]])

Position Encoding:
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  9.5042e-01,  9.9833e-02,  9.9950e-01,  9.9998e-03,
          9.9999e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01,  8.0658e-01,  1.9867e-01,  9.9800e-01,  1.9999e-02,
          9.9998e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01,  5.8275e-01,  2.9552e-01,  9.9550e-01,  2.9995e-02,
          9.9995e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01,  3.0114e-01,  3.8942e-01,  9.9201e-01,  3.9989e-02,
          9.9992e-01,  4.0000e-03,  1.0000e+00]])


In [8]:
# 给样本加上PE，利用广播
train_input_embedding += pe_mat;
train_label_embedding += pe_mat;

print(f"加上位置编码后的训练集输入Embedding：\n{train_input_embedding}")

加上位置编码后的训练集输入Embedding：
tensor([[[ 1.4604,  0.5658, -1.2865, -0.6023,  1.0133,  0.1371, -2.4131,
           1.6980],
         [ 0.4453,  1.8169,  2.1805,  0.1130, -0.3665,  1.5652, -0.0433,
           0.9233],
         [ 0.2141,  0.7359, -1.5642,  1.1986,  1.5558,  2.1774,  0.3687,
           1.2336],
         [ 1.9017,  0.7090,  0.4141,  0.6208, -1.8414,  2.4623,  0.1621,
          -0.6482],
         [-1.5084,  0.1490, -0.4471,  0.4145, -0.3057,  0.1838,  1.6673,
           2.0636]],

        [[ 1.7606,  1.1262,  0.1185,  0.6253, -1.8714,  2.4624,  0.1591,
          -0.6482],
         [ 0.4453,  1.8169,  2.1805,  0.1130, -0.3665,  1.5652, -0.0433,
           0.9233],
         [ 0.5131,  1.6731,  2.2793,  0.1115, -0.3565,  1.5652, -0.0423,
           0.9233],
         [-0.6105,  0.4306, -0.5410,  0.4179, -0.3157,  0.1838,  1.6663,
           2.0636],
         [-1.5084,  0.1490, -0.4471,  0.4145, -0.3057,  0.1838,  1.6673,
           2.0636]]], grad_fn=<AddBackward0>)


Step 5: 编码器 - Scaled Dot-Product Attetion

注意力机制就是计算Q和K的相似度作为一个权重，然后把这个权重赋予V，从而得到注意力，最后加到原特征上去，得到一个新特征。
这个新的特征相比于原特征，结合了与K的联系。

$$
\text{Attention}(Q, K, V)=\text{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$

Transform中使用的这个**Scaled Dot-Product Attetion**使用QK的点积作为相似度，除以一个$\sqrt{d_k}$就是scale操作，其目的是为了让softmax的方差不要那么大（想想softmax的形状和性质，方差太大会导致权重分布都集中到大的数字上去了）。

另外，Encoder部分还需要一个mask，其目的是消除之前pad出来的0，也就是把这些0的位置都遮掉。